In [1]:
import torch
import datasets, networks, sampling
import matplotlib.pyplot as plt
from torch import nn

In [2]:
#HYPERPARAMETERS
num_epochs = 15

# TODO Select a optimizer. [ 'adam', 'adamw', 'rmsprop' ]
optimizer_option = 'adam'

# TODO Select a batch size.
batch_size = 64

# TODO Select a learning rate.
lr = 0.000001


num_residual = 10
num_kernels = 64

In [3]:
ds = datasets.Dataset('mnist', batch_size=batch_size)
training_data = ds.get_train_data_loader()
test_data = ds.get_test_data_loader()

# ds.visualize_dataset(training_data)

In [4]:
# Training
import time
from torch.optim.lr_scheduler import StepLR, ExponentialLR, CosineAnnealingLR
from torch.optim import Adam, AdamW, RMSprop, SGD 
from torch.autograd import Variable

PixelCNN = networks.PixelCNN()
# Select device
# if torch.cuda.is_available():
#   device = torch.device('cuda:0')
# else:
#   device = torch.device('cpu')
device = torch.device('cuda:0')
print(f'Using device: {device}')
print('=========================================')

PixelCNN.to(device)

learning_rates = []
train_loss_curve = []
test_loss_curve = []
train_loss_epochs = []
test_loss_epochs = []
optimizer = None

# Select optimizer
if optimizer_option == "adam":
  optimizer = torch.optim.Adam(PixelCNN.parameters(), lr)
  
elif optimizer_option == "adamw":
  optimizer = torch.optim.AdamW(PixelCNN.parameters(), lr)
  
elif optimizer_option == "rmsprop":
  optimizer = torch.optim.RMSprop(PixelCNN.parameters(), lr)
  
else:
  optimizer = torch.optim.SGD(PixelCNN.parameters(), lr, momentum=0.9)  


criterion = nn.CrossEntropyLoss()


overall_start_time = time.time()

# training loop
for epoch in range(num_epochs):

  epoch_start_time = time.time()
  # Switch to training mode.
  PixelCNN.train()

  losses = []
  batch_idx = 0

  for images, labels in training_data:

    target = Variable(images[:,0,:,:]*255).long()
    images = images.to(device)
    target = target.to(device)


    optimizer.zero_grad()

    output = PixelCNN(images)

    # output = torch.reshape(output, (batch_size, 256, 28, 28))
    # output = output.permute(0,2,3,1).contiguous().view(-1, 256)

    loss = criterion(output, target)
    loss.backward()
    # torch.nn.utils.clip_grad_norm_(PixelCNN.parameters(), 1)
    optimizer.step()

    losses.append(loss.detach().clone())

    if batch_idx % 100 == 0:
      average_loss = torch.stack(losses).mean().item()
      losses = []
      print(f'Epoch: {epoch + 1:3d}/{num_epochs:3d}, Batch {batch_idx + 1:5d}, Loss: {average_loss:.4f}')
    batch_idx += 1

  epoch_end_time = time.time()
  print('-----------------------------------------')
  print(f'Epoch: {epoch + 1:3d} took {epoch_end_time - epoch_start_time:.2f}s')

  

  sampling.sample(PixelCNN, num_samples=5)

Using device: cuda:0
Epoch:   1/ 15, Batch     1, Loss: 5.7465
Epoch:   1/ 15, Batch   101, Loss: 4.8313
Epoch:   1/ 15, Batch   201, Loss: 4.3380
Epoch:   1/ 15, Batch   301, Loss: 3.8782
Epoch:   1/ 15, Batch   401, Loss: 3.3769
Epoch:   1/ 15, Batch   501, Loss: 2.8795
Epoch:   1/ 15, Batch   601, Loss: 2.4350
Epoch:   1/ 15, Batch   701, Loss: 2.0788
Epoch:   1/ 15, Batch   801, Loss: 1.7920
Epoch:   1/ 15, Batch   901, Loss: 1.5238
-----------------------------------------
Epoch:   1 took 63.60s
Epoch:   2/ 15, Batch     1, Loss: 1.3212
Epoch:   2/ 15, Batch   101, Loss: 1.2285
Epoch:   2/ 15, Batch   201, Loss: 1.0961
Epoch:   2/ 15, Batch   301, Loss: 1.0101
Epoch:   2/ 15, Batch   401, Loss: 0.9628
Epoch:   2/ 15, Batch   501, Loss: 0.9313
Epoch:   2/ 15, Batch   601, Loss: 0.9077
Epoch:   2/ 15, Batch   701, Loss: 0.8921
Epoch:   2/ 15, Batch   801, Loss: 0.8788
Epoch:   2/ 15, Batch   901, Loss: 0.8732
-----------------------------------------
Epoch:   2 took 62.70s
Epoch:   

KeyboardInterrupt: 